# KOSPI는 다른 차트의 변동성과 어떤 연관이 있을까?

## 데이터 가져오기

In [ ]:
# !pip install finance-datareader

In [ ]:
#프리셋처럼 그냥 통으로 때려박아놓기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN

import FinanceDataReader as fdr

In [ ]:
kospi = fdr.DataReader('KS11')
kospi["Change"] = kospi["Change"] * 100
kospi.drop(["Open", "High", "Low", "Close", "Volume", "UpDown", "Comp", "Amount", "MarCap"], axis=1, inplace=True)
kospi.rename(columns={"Change" : "KOSPI change"}, inplace=True)

In [ ]:
kospi.head(1)

In [ ]:
sp500 = fdr.DataReader('S&P500', "2001-06-08")
sp500["Change"] = round(sp500["Adj Close"].pct_change() * 100, 2)
sp500.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1, inplace=True)
sp500.drop("2001-06-08", axis=0, inplace=True)
sp500.rename(columns={"Change" : "S&P500 change"}, inplace=True)

In [ ]:
sp500.head(1)

In [ ]:
japan = fdr.DataReader('N225', "2001-06-08")
japan["Change"] = round(japan["Adj Close"].pct_change() * 100, 2)
japan.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1, inplace=True)
japan.drop("2001-06-08", axis=0, inplace=True)
japan.rename(columns={"Change" : "Nikkei change"}, inplace=True)

In [ ]:
japan.head(1)

In [ ]:
dollar = fdr.DataReader('USD/KRW', "2001-06-08")
dollar["Change"] = round(dollar["Adj Close"].pct_change() * 100, 2)
dollar.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1, inplace=True)
dollar.rename(columns={"Change" : "USD/KRW"}, inplace=True)

In [ ]:
dollar.head(1)

In [ ]:
us10 = fdr.DataReader('US10YT', "2001-06-08")
us10["Change"] = round(us10["Adj Close"].pct_change() * 100, 2)
us10.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1, inplace=True)
us10.drop("2001-06-08", axis=0, inplace=True)
us10.drop("2001-06-10", axis=0, inplace=True)
us10.rename(columns={"Change" : "US 10y change"}, inplace=True)

In [ ]:
us10.head(1)

In [ ]:
nasdaq = fdr.DataReader("IXIC", "2001-06-08")
nasdaq["Change"] = round(nasdaq["Adj Close"].pct_change() * 100, 2)
nasdaq.drop(["Open", "High", "Low", "Close", "Adj Close", "Volume"], axis=1, inplace=True)
nasdaq.drop("2001-06-08", axis=0, inplace=True)
nasdaq.rename(columns={"Change" : "NASDAQ change"}, inplace=True)

In [ ]:
nasdaq

## 병합

In [ ]:
df = pd.concat([kospi, sp500, japan, dollar, us10, nasdaq], axis=1)
df.drop("2024-01-24",axis=0, inplace=True)

#날짜가 없는 NaN을 어떻게 할 것인가?
#평균값을 구해서 때려넣기
df.fillna(df.mean(), inplace=True)
df = round(df,2) #DataFrame인 df의 모든 값을 소수점 둘째 자리까지 반올림

In [ ]:
df

## K-Mean을 사용한 군집 분류

In [ ]:
X = df[["S&P500 change","Nikkei change","USD/KRW","US 10y change","NASDAQ change"]]
y = df["KOSPI change"]

In [ ]:
inertias = []
for k in range(1,10):
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(X)
    inertias.append(model.inertia_)

In [ ]:
plt.plot(range(1,10), inertias, '-o')
plt.xlabel("클러스터 갯수")
plt.ylabel("inertias")
plt.xticks(range(1,10))
plt.show()

In [ ]:
#K=3을 잡아보자!
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
y_kmeans

In [ ]:
from sklearn.manifold import TSNE #차원 축소

In [ ]:
n_sne = X.shape[0]
tsne = TSNE(n_components=2) #2차원에 도식화
tsne_results = tsne.fit_transform(X)

In [ ]:
# plt.scatter(tsne_results[:,0], tsne_results[:,1], c=y_kmeans, cmap="plasma")
sns.scatterplot(x=tsne_results[:,0], y=tsne_results[:,1], data=y, hue=y_kmeans)